<a href="https://colab.research.google.com/github/Allenyang2/PytorchFL/blob/main/Flwr_Federated_Learning-v6-8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy
numpy.__version__

'1.22.4'

In [2]:
!python --version

Python 3.10.11


In [3]:
!pip install -q flwr[simulation] torch torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 13.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.0

In [4]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import sys
import random
import flwr as fl
from flwr.common import Metrics

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 2.0.1+cu118 and Flower 1.4.0


It is possible to switch to a runtime that has GPU acceleration enabled (on 

*   List item
*   List item

Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

In [5]:
CLASSES = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

We simulate having multiple datasets from multiple organizations (also called the "cross-silo" setting in federated learning) by splitting the original CIFAR-10 dataset into multiple partitions. Each partition will represent the data from a single organization. We're doing this purely for experimentation purposes, in the real world there's no need for data splitting because each organization already has their own data (so the data is naturally partitioned).

Each organization will act as a client in the federated learning system. So having ten organizations participate in a federation means having ten clients connected to the federated learning server:


In [6]:
NUM_CLIENTS = 10


Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap the resulting partitions by creating a PyTorch `DataLoader` for each of them:

In [7]:
# BATCH_SIZE = 32


# def load_datasets():
#     # Download and transform CIFAR-10 (train and test)
#     transform = transforms.Compose(
#         [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
#     )
#     trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
#     testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

#     # Split training set into 10 partitions to simulate the individual dataset
#     partition_size = len(trainset) // NUM_CLIENTS
#     lengths = [partition_size] * NUM_CLIENTS
#     datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

#     # Split each partition into train/val and create DataLoader
#     trainloaders = []
#     valloaders = []
#     for ds in datasets:
#         len_val = len(ds) // 10  # 10 % validation set
#         len_train = len(ds) - len_val
#         lengths = [len_train, len_val]
#         ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
#         trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
#         valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
#     testloader = DataLoader(testset, batch_size=BATCH_SIZE)
#     return trainloaders, valloaders, testloader


# trainloaders, valloaders, testloader = load_datasets()

In [8]:
BATCH_SIZE = 32
import random

def load_datasets(strategy: str = 'equal') -> Tuple[List[DataLoader], List[DataLoader], DataLoader]:

    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), 
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    if strategy == 'equal':
        # Split training set into 10 partitions to simulate the individual dataset
        partition_size = len(trainset) // NUM_CLIENTS
        lengths = [partition_size] * NUM_CLIENTS
        datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))
    elif strategy == 'non-iid':
        # Select random classes for each client (non-iid)
        labels = np.array(trainset.targets)
        idx = np.arange(len(trainset))
        class_indices = [idx[labels == i] for i in range(len(CLASSES))]
        datasets = []
        for i in range(NUM_CLIENTS):
            # Randomly choose classes for this client
            classes = np.random.choice(len(CLASSES), size=5, replace=False)
            # Select a fixed number of samples from each class
            samples = [class_indices[c][i * 100:(i + 1) * 100] for c in classes]
            samples = np.concatenate(samples)
            dataset = torch.utils.data.Subset(trainset, samples)
            datasets.append(dataset)
    elif strategy == 'random-percentage':
        # Randomly divide training data into NUM_CLIENTS subsets with different class distributions
        # Divide the training set randomly with different percentage distribution of classes
        class_count = {i: 0 for i in range(len(CLASSES))}
        for i in range(len(trainset)):
            label = trainset[i][1]
            class_count[label] += 1
        partitions = [[] for _ in range(NUM_CLIENTS)]
        for i in range(len(CLASSES)):
            class_indices = [j for j in range(len(trainset)) if trainset[j][1] == i]
            random.shuffle(class_indices)
            partition_sizes = []
            for k in range(NUM_CLIENTS):
                if k == NUM_CLIENTS - 1:
                    partition_sizes.append(len(class_indices) - sum(partition_sizes))
                else:
                    partition_sizes.append(random.randint(0, len(class_indices) - sum(partition_sizes)))
            partition_start = 0
            for k in range(NUM_CLIENTS):
                partition = class_indices[partition_start:partition_start+partition_sizes[k]]
                partitions[k].extend(partition)
                partition_start += partition_sizes[k]
        datasets = [torch.utils.data.Subset(trainset, partition) for partition in partitions]
    elif strategy == 'random-size':
        # Generate a list of random numbers that add up to the total number of samples minus NUM_CLIENTS
        # This ensures that each client gets at least one sample
        sizes = np.random.dirichlet(np.ones(NUM_CLIENTS), size=1) * (len(trainset) - NUM_CLIENTS)
        #https://numpy.org/doc/stable/reference/random/generated/numpy.random.dirichlet.html
        sizes = np.rint(sizes)  # Round to the nearest integers
        sizes = list(map(int, sizes.flatten()))  # Convert to a list of integers
        sizes = [size + 1 for size in sizes]  # Add one to each size to ensure each client gets at least one sample
        sizes[-1] = len(trainset) - np.sum(sizes[:-1])  # Ensure the sizes add up to the total number of samples

        # Shuffle the data indices
        indices = list(range(len(trainset)))
        random.shuffle(indices)

        # Partition the data into random sizes for each client
        datasets = []
        start = 0
        for size in sizes:
            subset_indices = indices[start: start + size]
            subset = torch.utils.data.Subset(trainset, subset_indices)
            datasets.append(subset)
            start += size
    else:
        raise ValueError(f"Invalid strategy: {strategy}")

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets()

100%|██████████| 170498071/170498071 [00:06<00:00, 28234362.07it/s]


Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


We now have a list of ten training sets and ten validation sets (`trainloaders` and `valloaders`) representing the data of ten different organizations. Each `trainloader`/`valloader` pair contains 4500 training examples and 500 validation examples. There's also a single `testloader` (we did not split the test set). Again, this is only necessary for building research or educational systems, actual federated learning systems have their data naturally distributed across multiple partitions.

Let's take a look at the first batch of images and labels in the first training set (i.e., `trainloaders[0]`) before we move on:

In [9]:
images, labels = next(iter(trainloaders[0]))

# Reshape and convert images to a NumPy array
# matplotlib requires images with the shape (height, width, 3)
images = images.permute(0, 2, 3, 1).numpy()
# Denormalize
images = images / 2 + 0.5

# Create a figure and a grid of subplots
# fig, axs = plt.subplots(4, 8, figsize=(12, 6))

# # Loop over the images and plot them
# for i, ax in enumerate(axs.flat):
#     ax.imshow(images[i])
#     ax.set_title(CLASSES[labels[i]])
#     ax.axis("off")

# # Show the plot
# fig.tight_layout()
# plt.show()

### Defining the model

We use the simple CNN described in the [PyTorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#define-a-convolutional-neural-network):

In [10]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Let's continue with the usual training and test functions:

In [11]:
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

If 'non-iid' is chosen, it generates non-iid data distribution for the clients. Specifically, for each client, it randomly chooses 5 out of 10 classes, and samples 100 images from each of those classes. This creates a situation where each client has a different distribution of classes, and some classes may be completely missing from some clients. This is just one possible way to create non-iid data, and you can modify it to suit your specific use case.

Note that the non-iid strategy can result in some clients having very little data, especially if they happen to get classes with very few samples. This can affect the learning performance, and


If 'random-percentage" is chosen then randomly generate a distribution of class counts (with each count representing the number of samples of a particular class that the client should have), and then use this distribution to randomly select samples from the training set. The class_counts list contains the number of samples for each class in the entire training set, and the class_distribution list contains a randomly generated distribution of class counts for each client. We then use the % operator to determine which class a particular sample belongs to, and use the corresponding distribution value to 
decide whether to include it in the client's dataset.

This implementation uses the 'random' strategy to randomly divide the training set into NUM_CLIENTS partitions with different percentage distribution of classes. The implementation first counts the number of samples for each class in the training set. Then, for each class, it randomly partitions the samples for that class into NUM_CLIENTS partitions, where each partition has a different percentage distribution of samples from that class. Finally, the implementation creates a Subset object for each partition and returns the list of these objects as datasets. The rest of the implementation is the same as the original load_datasets function.

### Training the model

We now have all the basic building blocks we need: a dataset, a model, a training function, and a test function. Let's put them together to train the model on the dataset of one of our organizations (`trainloaders[0]`). This simulates the reality of most machine learning projects today: each organization has their own data and trains models only on this internal data: 

In [12]:
trainloader = trainloaders[0]
valloader = valloaders[0]
net = Net().to(DEVICE)

# for epoch in range(5):
#     train(net, trainloader, 1)
#     loss, accuracy = test(net, valloader)
#     print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")

# loss, accuracy = test(net, testloader)
# print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

Training the simple CNN on our CIFAR-10 split for 5 epochs should result in a test set accuracy of about 41%, which is not good, but at the same time, it doesn't really matter for the purposes of this tutorial. The intent was just to show a simplistic centralized training pipeline that sets the stage for what comes next - federated learning!

### Updating model parameters

In federated learning, the server sends the global model parameters to the client, and the client updates the local model with the parameters received from the server. It then trains the model on the local data (which changes the model parameters locally) and sends the updated/changed model parameters back to the server (or, alternatively, it sends just the gradients back to the server, not the full model parameters).

We need two helper functions to update the local model with parameters received from the server and to get the updated model parameters from the local model: `set_parameters` and `get_parameters`. The following two functions do just that for the PyTorch model above.

The details of how this works are not really important here (feel free to consult the PyTorch documentation if you want to learn more). In essence, we use `state_dict` to access PyTorch model parameter tensors. The parameter tensors are then converted to/from a list of NumPy ndarray's (which Flower knows how to serialize/deserialize):

In [13]:
def get_parameters(net) -> List[np.ndarray]:
    parameters = [val.cpu().numpy() for _, val in net.state_dict().items()]
    # print(f"Parameters size: {get_params_size(parameters)} bytes")
    return parameters


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

### Implementing a Flower client

With that out of the way, let's move on to the interesting part. Federated learning systems consist of a server and multiple clients. In Flower, we create clients by implementing subclasses of `flwr.client.Client` or `flwr.client.NumPyClient`. We use `NumPyClient` in this tutorial because it is easier to implement and requires us to write less boilerplate.

To implement the Flower client, we create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`:

* `get_parameters`: Return the current local model parameters
* `fit`: Receive model parameters from the server, train the model parameters on the local data, and return the (updated) model parameters to the server
* `evaluate`: Receive model parameters from the server, evaluate the model parameters on the local data, and return the evaluation result to the server

We mentioned that our clients will use the previously defined PyTorch components for model training and evaluation. Let's see a simple Flower client implementation that brings everything together:

In [14]:
def get_params_size(parameters):
    total_size = 0
    for p in parameters:
        total_size += sys.getsizeof(p)
        if isinstance(p, np.ndarray):
            total_size += p.nbytes
    return total_size


In [15]:
import time
# import logging
# import multiprocessing

class FlowerClient(fl.client.NumPyClient):

      def __init__(self, cid, net, trainloader, valloader):
          self.cid = cid
          self.net = net
          self.trainloader = trainloader
          self.valloader = valloader
          # self.logger = logging.getLogger(f"Client {cid}")
          # self.logger.setLevel(logging.INFO)
          # self.file_handler = logging.FileHandler(f"client_{cid}_log.txt")
          # self.logger.addHandler(self.file_handler)

      def get_parameters(self, config):
          return get_parameters(self.net)

      def fit(self, parameters, config):
        with open(f"client_{self.cid}_log.txt", "a") as f:
          set_parameters(self.net, parameters)
          start_time = time.time()
          train(self.net, self.trainloader, epochs=2)
          end_time = time.time()
          total_time = end_time - start_time
          adj_total_time = ("%.3f" %(total_time))
          parameters = get_parameters(self.net)
          print(f"Client ID: {self.cid}, Parameters size: {get_params_size(parameters)} bytes, Total Time: {adj_total_time}")
          f.write(f"Client ID: {self.cid}, Parameters size: {get_params_size(parameters)} bytes, Total Time: {adj_total_time}\n")
          f.write("\n")

          with open("param_output.txt", "w") as f:

            print("Sending back new parameters to the global model:", parameters, file=f)
          return parameters, len(self.trainloader), {}

      def evaluate(self, parameters, config):
        # with open(f"client_{self.cid}_evaluate.txt", "w") as d:
        with open(f"client_{self.cid}_evaluate.txt", "a") as f:
          set_parameters(self.net, parameters)
          loss, accuracy = test(self.net, self.valloader)
          print(f"Client ID: {self.cid}, Loss: {loss}, Accuracy: {accuracy}")
          f.write(f"Client ID: {self.cid}, Loss: {loss}, Accuracy: {accuracy}")
          f.write("\n")
          return float(loss), len(self.valloader), {"accuracy": float(accuracy)}



Our class `FlowerClient` defines how local training/evaluation will be performed and allows Flower to call the local training/evaluation through `fit` and `evaluate`. Each instance of `FlowerClient` represents a *single client* in our federated learning system. Federated learning systems have multiple clients (otherwise, there's not much to federate), so each client will be represented by its own instance of `FlowerClient`. If we have, for example, three clients in our workload, then we'd have three instances of `FlowerClient`. Flower calls `FlowerClient.fit` on the respective instance when the server selects a particular client for training (and `FlowerClient.evaluate` for evaluation).

### Using the Virtual Client Engine

In this notebook, we want to simulate a federated learning system with 10 clients on a single machine. This means that the server and all 10 clients will live on a single machine and share resources such as CPU, GPU, and memory. Having 10 clients would mean having 10 instances of `FlowerClient` in memory. Doing this on a single machine can quickly exhaust the available memory resources, even if only a subset of these clients participates in a single round of federated learning.

In addition to the regular capabilities where server and clients run on multiple machines, Flower, therefore, provides special simulation capabilities that create `FlowerClient` instances only when they are actually necessary for training or evaluation. To enable the Flower framework to create clients when necessary, we need to implement a function called `client_fn` that creates a `FlowerClient` instance on demand. Flower calls `client_fn` whenever it needs an instance of one particular client to call `fit` or `evaluate` (those instances are usually discarded after use, so they should not keep any local state). Clients are identified by a client ID, or short `cid`. The `cid` can be used, for example, to load different local data partitions for different clients, as can be seen below:

In [16]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model

    # # List of acceptable cids
    # acceptable_cids = ["10", "2", "4", "7", "8", "9"]  # replace with your cids

    # # Check if provided cid is acceptable
    # if cid not in acceptable_cids:
    #     raise ValueError(f"Unrecognized cid: {cid}")


    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

### Starting the training

We now have the class `FlowerClient` which defines client-side training/evaluation and `client_fn` which allows Flower to create `FlowerClient` instances whenever it needs to call `fit` or `evaluate` on one particular client. The last step is to start the actual simulation using `flwr.simulation.start_simulation`. 

The function `start_simulation` accepts a number of arguments, amongst them the `client_fn` used to create `FlowerClient` instances, the number of clients to simulate (`num_clients`), the number of federated learning rounds (`num_rounds`), and the strategy. The strategy encapsulates the federated learning approach/algorithm, for example, *Federated Averaging* (FedAvg).

Flower has a number of built-in strategies, but we can also use our own strategy implementations to customize nearly all aspects of the federated learning approach. For this example, we use the built-in `FedAvg` implementation and customize it using a few basic parameters. The last step is the actual call to `start_simulation` which - you guessed it - starts the simulation:

In [17]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [18]:
#function to remove old files (possibly for colab only*)

import os
import glob

file_key = "client*.txt"
for del_file in glob.glob(file_key):
  os.remove(del_file)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

The only thing left to do is to tell the strategy to call this function whenever it receives evaluation metric dictionaries from the clients:

In [19]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.8,
    fraction_evaluate=0.5,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=10,
    evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=9),
    strategy=strategy,
    client_resources=client_resources,
)



INFO flwr 2023-06-08 14:44:58,999 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=9, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=9, round_timeout=None)
2023-06-08 14:45:02,797	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-06-08 14:45:04,918 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'GPU': 1.0, 'memory': 7861850112.0, 'object_store_memory': 3930925056.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'GPU': 1.0, 'memory': 7861850112.0, 'object_store_memory': 3930925056.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-06-08 14:45:04,923 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-08 14:45:04,931 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
INFO flwr 2023-06-08 14:45:11,986 | se

(launch_and_fit pid=1991) Client ID: 7, Parameters size: 249288 bytes, Total Time: 5.238
(launch_and_fit pid=2083) Client ID: 8, Parameters size: 249288 bytes, Total Time: 4.331
(launch_and_fit pid=2164) Client ID: 2, Parameters size: 249288 bytes, Total Time: 3.480
(autoscaler +1m20s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +1m20s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(launch_and_fit pid=2256) Client ID: 9, Parameters size: 249288 bytes, Total Time: 3.483


ERROR flwr 2023-06-08 14:46:22,060 | ray_client_proxy.py:87 | ray::launch_and_fit() (pid=2349, ip=172.28.0.12)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 147, in launch_and_fit
    client: Client = _create_client(client_fn, cid)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 168, in _create_client
    client_like: ClientLike = client_fn(cid)
  File "<ipython-input-16-d6f950038e92>", line 11, in client_fn
ValueError: Unrecognized cid: 5
ERROR:flwr:ray::launch_and_fit() (pid=2349, ip=172.28.0.12)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 147, in launch_and_fit
    client: Client = _create_client(client_fn, cid)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 168, in _create_client
    client_like: ClientLike = client_fn(cid)
  File "<ipython-input-16-d

(launch_and_fit pid=2540) Client ID: 4, Parameters size: 249288 bytes, Total Time: 3.547


ERROR flwr 2023-06-08 14:46:52,702 | ray_client_proxy.py:87 | ray::launch_and_fit() (pid=2630, ip=172.28.0.12)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 147, in launch_and_fit
    client: Client = _create_client(client_fn, cid)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 168, in _create_client
    client_like: ClientLike = client_fn(cid)
  File "<ipython-input-16-d6f950038e92>", line 11, in client_fn
ValueError: Unrecognized cid: 6
ERROR:flwr:ray::launch_and_fit() (pid=2630, ip=172.28.0.12)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 147, in launch_and_fit
    client: Client = _create_client(client_fn, cid)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 168, in _create_client
    client_like: ClientLike = client_fn(cid)
  File "<ipython-input-16-d

...

(launch_and_evaluate pid=3010) Client ID: 7, Loss: 0.061265372514724734, Accuracy: 0.34


ERROR flwr 2023-06-08 14:47:51,894 | ray_client_proxy.py:87 | ray::launch_and_fit() (pid=3124, ip=172.28.0.12)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 147, in launch_and_fit
    client: Client = _create_client(client_fn, cid)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 168, in _create_client
    client_like: ClientLike = client_fn(cid)
  File "<ipython-input-16-d6f950038e92>", line 11, in client_fn
ValueError: Unrecognized cid: 1
ERROR:flwr:ray::launch_and_fit() (pid=3124, ip=172.28.0.12)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 147, in launch_and_fit
    client: Client = _create_client(client_fn, cid)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 168, in _create_client
    client_like: ClientLike = client_fn(cid)
  File "<ipython-input-16-d

(autoscaler +3m11s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0, 'GPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(launch_and_fit pid=3186) Client ID: 8, Parameters size: 249288 bytes, Total Time: 3.498
(launch_and_fit pid=3280) Client ID: 9, Parameters size: 249288 bytes, Total Time: 4.597
(launch_and_fit pid=3366) Client ID: 7, Parameters size: 249288 bytes, Total Time: 4.308
(launch_and_fit pid=3456) Client ID: 4, Parameters size: 249288 bytes, Total Time: 3.494
(launch_and_fit pid=3544) Client ID: 2, Parameters size: 249288 bytes, Total Time: 3.567


ERROR flwr 2023-06-08 14:48:57,481 | ray_client_proxy.py:87 | ray::launch_and_fit() (pid=3633, ip=172.28.0.12)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 147, in launch_and_fit
    client: Client = _create_client(client_fn, cid)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 168, in _create_client
    client_like: ClientLike = client_fn(cid)
  File "<ipython-input-16-d6f950038e92>", line 11, in client_fn
ValueError: Unrecognized cid: 0
ERROR:flwr:ray::launch_and_fit() (pid=3633, ip=172.28.0.12)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 147, in launch_and_fit
    client: Client = _create_client(client_fn, cid)
  File "/usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_client_proxy.py", line 168, in _create_client
    client_like: ClientLike = client_fn(cid)
  File "<ipython-input-16-d

KeyboardInterrupt: ignored

In [ ]:

file_key = "client*.txt"
file_name = "collect_logs"
input_file_name = file_name + '.txt'
with open(input_file_name, "w") as outfile:
  for file in glob.glob(file_key):
    with open(file, 'r') as infile:
      outfile.write(infile.read())
      outfile.write("\n")
